# DeepDream
*
DeepDream uses a convolutional neural network to find and enhance patterns in images via algorithmic pareidolia, thus creating a dream-like hallucinogenic appearance in the deliberately over-processed images [(wiki)](https://en.wikipedia.org/wiki/DeepDream). *

In this kernel I will explore DeepDream by using a both an InceptionV3 and a VGG-16 pretrained model in Keras. The choice of your convnet will affect your visualisation, as different architectures result in different learned features. Inception for example has been trained on (amongst others) many images of animals and the use of this convnet in a DeepDream often outputs pictures with a lot of eye-like features. By playing around with the different architectures, I will try to understand more on how they work. 

Inspiration for this kernel were kernels by [Carlo Alberto](https://www.kaggle.com/carloalbertobarbano/convolutional-network-visualizations-deep-dream/notebook) and by [Paul Mooney](https://www.kaggle.com/paultimothymooney/pre-trained-pytorch-monkeys-a-deep-dream), which both use PyTorch .  Most of the code is from keras/deepdream found on the [github](https://github.com/keras-team/keras/blob/master/examples/deep_dream.py) of the Keras Team. I've also used the book Deep Learning with Python by François Chollet, which gives insights on how the DeepDream algorithm (and deep learning in general) works.

In the case of DeepDream an arbitrary image is fed to the network and is analyzed. The activation of an layer is maximized and the networks is asked to enhance whatever it detected. Each layer of the network deals with features at a different level of abstraction, so the complexity of features we generate depends on which layer we choose to enhance ([source](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html )).


In short: 

- Load the original image.
- Define a number of processing scales (i.e. image shapes), from smallest to largest.
- Resize the original image to the smallest scale.
- For every scale, starting with the smallest (i.e. current one):
    - Run gradient ascent
    - Upscale image to the next scale
    - Reinject the detail that was lost at upscaling time
- Stop when we are back to the original size.

To obtain the detail lost during upscaling, we simply take the original image, shrink it down, upscale it, and compare the result to the (resized) original image.


In [ ]:
import numpy as np
import scipy
import PIL.Image

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications import inception_v3
from keras import backend as K

# To fix FailedPreconditionError:
sess = tf.InteractiveSession()
with tf.Session() as sess:
     sess.run(tf.global_variables_initializer()) 

## InceptionV3

When Google released DeepDream, it was based on an Inception model - now famous for its psychedelic output. Let's start with an InceptionV3. 

First, we are not going to train the model ourselves, so we want to disable all training specific operations. Then the model can be loaded. Note that this builds the Inception V3 network without its convolutional base. A dictionary is set with the names of the layers for which we try to maximize activation, as well as their weight in the final loss we try to maximize. Then we set a function to load, resize and convert the image and a function for the opposite. 

In [ ]:
# Disable all training specific operations
K.set_learning_phase(0)


# The model will be loaded with pre-trained inceptionv3 weights.
model = inception_v3.InceptionV3(weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                 include_top=False)
dream = model.input
print('Model loaded.')


# You can tweak these setting to obtain new visual effects.
settings = {
    'features': {
        'mixed2': 0.2,
        'mixed3': 0.5,
        'mixed4': 2.,
        'mixed5': 1.5,
    },
}


# Set a function to load, resize and convert the image. 
def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


# And a function to do the opposite: convert a tensor into an image. 
def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# Set a dictionary that maps the layer name to the layer instance. 
# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])


# Define the loss. The way this works is first the scalar variable *loss* is set. 
# Then the loss will be defined by adding layer contributions to this variable. 
loss = K.variable(0.)

for layer_name in settings['features']:
    # Add the L2 norm of the features of a layer to the loss.
    assert (layer_name in layer_dict.keys(),
            'Layer ' + layer_name + ' not found in model.')
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    if K.image_data_format() == 'channels_first':
        loss += coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
    else:
        loss += coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

In order to do a gradient-ascent with respect to the loss, we compute the gradients, normalize and create a function for this process (so given an image, retrieve the value of the loss and gradients). We define the gradient ascent function over a number of iterations. 

In [ ]:
# Compute the gradients of the dream wrt the loss.
grads = K.gradients(loss, dream)[0]
# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())

# Set up function to retrieve the value of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# Helper funtion to resize
def resize_img(img, size):
    img = np.copy(img)
    if K.image_data_format() == 'channels_first':
        factors = (1, 1,
                   float(size[0]) / img.shape[2],
                   float(size[1]) / img.shape[3])
    else:
        factors = (1,
                   float(size[0]) / img.shape[1],
                   float(size[1]) / img.shape[2],
                   1)
    return scipy.ndimage.zoom(img, factors, order=1)


# Define the gradient ascent function over a number of iterations. 
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('..Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x


# Set hyperparameters. The ocatave_scale is the ratio between each successive scale (remember the upscaling mentioned before?). 
# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.008  # Gradient ascent step size
num_octave = 5  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale
max_loss = 10.

Choose an image and show. Let's take an surrealistic painting by Dali and see if the weirdness can be turned up a notch..

In [ ]:
base_image_path = "../input/painter-by-numbers/train_2/21500.jpg"
img = PIL.Image.open(base_image_path)
img


And then reshape, resize and process with the help of the functions above. 

In [ ]:
img = preprocess_image(base_image_path)
if K.image_data_format() == 'channels_first':
    original_shape = img.shape[2:]
else:
    original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)

save_img('dream.jpg',deprocess_image(np.copy(img)))

Let's check out the first 'dream'.  There are already some eye-like features popping up, and it looks a lot more psychedelic than before, but it's not the nicest dream... 

In [ ]:
dreamout = PIL.Image.open('dream.jpg')
dreamout

## Next up

So this was the first image. In next versions I will check different hyperparameters, use the VGG16 and take a look at combining images (Neural style transfer). Also I will check out some smaller images, as the InceptionV3 is trained on images smaller than the one used here and this might have some effect. Stay tuned. 